In [1]:
#!git clone https://github.com/modusdatascience/choldate.git
#!cd choldate; python3 setup.py install 
#!pip install p_tqdm
import sys
sys.path.append('/content/choldate')
from choldate import cholupdate, choldowndate
from scipy.linalg import solve_triangular
import scipy
import multiprocessing
from multiprocessing import Pool, cpu_count
#from p_tqdm import p_map, p_starmap
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats 
from scipy.stats.distributions import chi2
from tqdm.notebook import tqdm
import itertools
import seaborn as sns
import pandas as pd
from scipy.special import softmax
from scipy.special import logsumexp
import time
from itertools import repeat
from functools import partial

In [2]:
def get_moments(X):
  n, p = X.shape
  mu_hat = np.mean(X, axis=0)
  Sigma_hat = 1/n * (X - mu_hat).T @ (X - mu_hat)
  return mu_hat, Sigma_hat

def complement(n, idxs):
  if len(idxs) == 0:
    return np.arange(n)
  return np.delete(np.arange(n), idxs)

def scale_cov(Sigma, s):
  s = np.sqrt(s)
  return s[None, :] * Sigma * s[:, None]

def standardize_cov(Sigma):
  s = np.diag(Sigma)**(-1)
  return scale_cov(Sigma, s)

def random_argmin(x):
  return np.random.choice(np.flatnonzero(x == x.min()))

def quadratic_form(X, A, Y):
  return np.sum((X @ A) * Y , axis=1)

def diagonal_left_multiply(D, A):
  return D[:, None] * A

def diagonal_right_multiply(A, D):
  return A * D[None, :]

def update_cholesky_after_removing_first(L):
  L_ = L[1:, 1:].copy()
  v = L[1:, 0].copy()
  L_ = L_.T
  cholupdate(L_, v)
  return L_.T

def update_cholesky_after_removing_last(L):
  return L[:L.shape[0] - 1, :L.shape[1] - 1]

def update_cholesky_after_adding_last(L, v):
  p = len(v)
  if p == 1:
    return np.sqrt(np.array([v]))
  a = v[:(p-1)]
  d = v[p-1]
  a_ = solve_triangular(L, a, lower=True) 
  d_ = np.sqrt(d - np.inner(a_, a_))
  L_ = np.zeros((p, p))
  L_[:(p-1), :(p-1)] = L
  L_[p-1, :(p-1)] = a_
  L_[p-1, p-1] = d_
  return L_

def solve_with_cholesky(L, v):
  return solve_triangular(L.T, solve_triangular(L, v, lower=True), lower=False)

In [3]:
class ColumnSubsetSelection():
  def __init__(self):
    self.colinearity_errors = {}

  def error_check(self, objective, method):

    if not isinstance(self.k, (int, np.integer)) or self.k <= 0 or self.k > self.p:
      raise ValueError("k must be an integer > 0 and <= p.")
    if objective not in {'css'}:
      raise ValueError("Requested objective not supported.")
    if method not in {'greedy', 'swap'}:
      raise ValueError("Requested method not supported.")

  def select_subset_from_data(self, X, k, standardize=False, center=True, objective='css', method='greedy', **kwargs):
    
    objective=objective.lower()
    method=method.lower()
    self.n, self.p = X.shape
    self.k = k
    self.error_check(objective, method)

    self.X = X
    mu = np.mean(self.X, axis = 0)

    if not center and not standardize: 
      Sigma = 1/self.n * self.X.T @ self.X
    if center and not standardize:
      self.X -= mu
      Sigma = 1/self.n * self.X.T @ self.X 
    if standardize and not center:
      # fix this 
      Sigma = 1/self.n * (self.X -  mu).T (self.X -  mu)
      self.X = diagonal_left_multiply(self.X - mu, 1/np.sqrt(np.diag(Sigma))) + mu
      Sigma = 1/self.n * self.X.T @ self.X
    if standardize and center:
      self.X -= mu
      self.X = diagonal_left_multiply(self.X, 1/np.sqrt(np.diag(Sigma)))
      Sigma = 1/self.n * self.X.T @ self.X 
    
    self.select_subset_from_cov(Sigma, k, standardize=False, objective=objective, method=method, from_data=True, **kwargs)

  def select_subset_from_cov(self, Sigma, k, standardize=False, objective='css', method='greedy', **kwargs):

    from_data = kwargs.get('from_data', False)
    full_rank = kwargs.get('full_rank', False)

    self.tol = kwargs.get('tol', 10e-12)

    if not from_data:
      if Sigma.shape[0] != Sigma.shape[1]:
        raise ValueError("Sigma must be a square matrix.")
      self.k = k
      self.p = Sigma.shape[0]
      objective=objective.lower()
      method=method.lower()
      self.error_check(objective, method)
    #should be under an else?
    if standardize:
      Sigma = standardize_cov(Sigma)
    
    self.Sigma = Sigma
    self.p = Sigma.shape[0]
    self.k = k

    max_iter = kwargs.get('max_iter', 100)
    S_init = kwargs.get('S_init', None)

    if objective == 'css':
      objective_func = self._CSS_objective_func
    if objective == 'pcss':
      objective_func = self._PCSS_objective_func
    if objective == 'fss':
      objective_func = self._FSS_objective_func

    if method == 'greedy':
      self._greedy_subset_selection(objective_func, full_rank=full_rank)
    if method == 'swap':
      self._swapping_subset_selection(objective_func, S_init=S_init, max_iter=max_iter, full_rank=full_rank)

  def _swap(self, Sigma, idxs1, idxs2, idx_order=None, disjoint=False, row=True, col=True):
    if not disjoint:
      idxs1, idxs2 = set(idxs1), set(idxs2)
      idxs1, idxs2 = list(idxs1 - idxs2) , list(idxs2 - idxs1)
    if len(idxs1) == 0:
      return
    self._perm(Sigma, np.concatenate([idxs1, idxs2]), np.concatenate([idxs2, idxs1]), idx_order=idx_order, row=True, col=True)
  
  def _perm(self, Sigma, idxs_to_apply, perm, idx_order=None, row=True, col=True):
    if col:
      Sigma[:, idxs_to_apply] = Sigma[:, perm]
    if row:
      Sigma[idxs_to_apply, :] = Sigma[perm, :]
    if idx_order is not None:
      idx_order[idxs_to_apply] = idx_order[perm]

  def _is_invertible(self, Sigma_S):
    try:
      Sigma_S_L = np.linalg.cholesky(Sigma_S)
    except:
      return False, None
    if self.k == 1 and Sigma_S[0, 0] > self.tol:
      return True, Sigma_S_L
    idxs = np.arange(self.k)
    perm_idxs = np.concatenate([np.arange(1, self.k), [0]])
    for i in range(self.k):
      Sigma_T_L = update_cholesky_after_removing_first(Sigma_S_L)
      v = Sigma_S[1:self.k, 0]
      if Sigma_S[0, 0] - v.T @ solve_with_cholesky(Sigma_T_L, v) < self.tol:
        return False, None
      self._perm(Sigma_S, idxs, perm_idxs) 
      Sigma_S_L = update_cholesky_after_adding_last(Sigma_T_L, Sigma_S[self.k-1, :])
    return True, Sigma_S_L
  
  def _regress_one_off(self, Sigma, j):
    Sigma[:, :] = Sigma - np.outer(Sigma[:, j], Sigma[:, j])/Sigma[j, j]    
  
  def _regress_off(self, Sigma, S):
    for j in S:
      self._regress_one_off(Sigma, j)

  def _CSS_objective_func(self, Sigma_R):
    return -1 * np.sum(np.square(Sigma_R), axis=1)/np.diag(Sigma_R)

  #def _colinearity_objective_func(self, Sigma_R):
  #  return np.diag(Sigma_R)

  def _greedy_subset_selection(self, objective_func, full_rank=False):
    
    self.terminated_early=False
    
    self.S_ordered = -1 * np.ones(self.k).astype(int)
    Sigma_R = self.Sigma.copy()
    self._idx_order = np.arange(self.p)
    num_active=self.p

    #Sigma_S_L = np.array([[]])

    for i in range(self.k):
      Sigma_R_active = Sigma_R[:num_active, :num_active]
      self._S_in = self.S_ordered[:i] # 
      objectives = objective_func(Sigma_R_active)
      j_star = random_argmin(objectives)
      S_new = self._idx_order[j_star]
      self.S_ordered[i] = S_new
      #Sigma_S_L = update_cholesky_after_adding_last(Sigma_S_L, self.Sigma[S_new, self.S_ordered[:(i+1)]])
      self._regress_one_off(Sigma_R_active, j_star)
      self._swap(Sigma_R, [j_star], [self.p - i - 1], idx_order=self._idx_order)
      num_active -= 1
      
      if not full_rank:
        zero_idxs = np.where(np.diag(Sigma_R_active)[:num_active] < self.tol)[0]
        num_zero_idxs = len(zero_idxs)
        idxs_to_swap = np.arange(num_active - num_zero_idxs, num_active)
        self._swap(Sigma_R, zero_idxs, idxs_to_swap, idx_order=self._idx_order)
        num_active -= num_zero_idxs
      
      if num_active == 0 and i != self.k - 1:
        self.terminated_early = True
        print('Terminating early: ' + str(self.S_ordered[:(i+1)]) + ' are sufficient to explain the remaining')
        break

    self._perm(Sigma_R, np.arange(self.p), np.argsort(self._idx_order))
    self.Sigma_R = Sigma_R
    self.S = set(self.S_ordered)
    #self._Sigma_S_inv = solve_with_cholesky(Sigma_S_L, np.eye(self.k)) 
    #self._perm(self._Sigma_S_inv, np.arange(self.k), np.argsort(self.S_ordered)) 


  def _swapping_subset_selection(self, objective_func, S_init=None, max_iter=100, full_rank=False):

    self.converged = False
    d = self.p - self.k
    self._idx_order = np.arange(self.p)
    
    if S_init is None:
      S_init = np.random.choice(self._idx_order, self.k, replace=False)
    elif len(S_init) != self.k:
      raise ValueError("Initial subset must be of length k.")
    
    self.S_init = np.array(list(S_init))
    Sigma_R = self.Sigma.copy()
    self._swap(Sigma_R, np.arange(d, self.p), self.S_init, idx_order=self._idx_order)
    S = self._idx_order[d:].copy()
    Sigma_S = self.Sigma[:, S][S, :].copy()
    invertible, Sigma_S_L = self._is_invertible(Sigma_S) 
    
    if not invertible:
      self.colinearity_errors[frozenset(S)] = None
      raise ValueError("Colinearity issue: covariance of initial selected subset is not full rank.")
    
    self._regress_off(Sigma_R, np.arange(d, self.p))
    zero_idxs = np.where(np.diag(Sigma_R)[:d] <= self.tol)[0]
    num_zero_idxs = len(zero_idxs)
    
    if full_rank and num_zero_idxs > 0:
      self.colinearity_errors[frozenset(S)] = frozenset(self._idx_order[zero_idxs])
      raise ValueError("Colinearity issue: the initial subset perfectly predicts some other features")

    N = 0
    not_replaced = 0
    subset_idxs = np.arange(d, self.p)
    subset_idxs_permuted = np.concatenate([subset_idxs[1:], np.array([subset_idxs[0]])])
    break_flag = False 

    while N < max_iter and (not break_flag):
      for i in range(self.k):
        S_0 = S[0]
        T = S[1:]
      
        Sigma_T_L = update_cholesky_after_removing_first(Sigma_S_L) 
        v = self.Sigma[:, S_0] - self.Sigma[:, T] @ solve_with_cholesky(Sigma_T_L, self.Sigma[T, S_0]) if self.k > 1 else self.Sigma[:, S_0]
        reordered_v = v[self._idx_order]

        Sigma_R = Sigma_R + np.outer(reordered_v, reordered_v)/v[S_0]
        self._swap(Sigma_R, np.array([0]), np.array([d]), idx_order=self._idx_order)  
            
        if not full_rank:
          zero_idxs = np.where(np.diag(Sigma_R)[:(d + 1)] <= self.tol)[0]
          num_zero_idxs = len(zero_idxs)
          self._swap(Sigma_R, zero_idxs, np.arange(d + 1 - num_zero_idxs, d + 1), idx_order=self._idx_order)
        else:
          num_zero_idxs = 0 
        num_active = d + 1 - num_zero_idxs 
        self._S_in = T
        objectives = objective_func(Sigma_R[:num_active, :num_active])
        choices = np.flatnonzero(objectives == objectives.min())

        if 0 in choices:
          not_replaced += 1
          j_star = 0 
        else:
          not_replaced = 0
          j_star = np.random.choice(choices)

        S_new = self._idx_order[j_star]
        S[:self.k-1] = S[1:]
        S[self.k-1] = S_new
        Sigma_S_L = update_cholesky_after_adding_last(Sigma_T_L, self.Sigma[S_new, S])
        self._regress_one_off(Sigma_R[:(d+1), :(d+1)], j_star) 
        self._swap(Sigma_R, np.array([j_star]), np.array([d]), idx_order=self._idx_order)
        self._perm(Sigma_R, subset_idxs,  subset_idxs_permuted, idx_order=self._idx_order)
        if not_replaced == self.k:
          self.converged=True
          break_flag=True
          break
      N += 1

    self._perm(Sigma_R, np.arange(self.p), np.argsort(self._idx_order))
    self.Sigma_R = Sigma_R
    self.S = set(S.astype(int))
    #self._Sigma_S_inv = solve_with_cholesky(Sigma_S_L, np.eye(self.k))
    #self._perm(self._Sigma_S_inv, np.arange(self.k), np.argsort(S))




    
    
    

In [4]:
def perm_3d_inplace(tensor, idxs_to_apply, perm):
  tensor[:, :, idxs_to_apply] = tensor[:, :, perm]
  tensor[:, idxs_to_apply, :] = tensor[:, perm, :]  

def generate_subset_factor_sample_cov(n, C_chol, W, D, rho=None, B=None, chi_sqrts=None, normals=None, S=None):
  squeeze = False
  if B is None:
    B = 1
    squeeze = True
  k = C_chol.shape[0]
  p = k + W.shape[0]
  D_sqrt = np.sqrt(D)
  if chi_sqrts is None:
    chis = np.random.chisquare(df=np.arange(n - 1, n - p - 1 , -1), size=(B,p))
    chi_sqrts = np.sqrt(chis)
  if normals is None:
    normals = np.random.normal(0, 1, (B, int(p*(p-1)/2)) )

  L_top_left = np.zeros((B, k, k))
  L_top_left[:, np.tri(k, dtype=bool, k=-1)] = normals[:, :int(k *(k-1)/2)]
  i, j = np.diag_indices(k)
  L_top_left[:, i, j] = chi_sqrts[:, :k]

  L_bottom_left = normals[:, int(k *(k-1)/2) : int(k *(k-1)/2) + (p-k)*k].reshape((B, p-k, k))

  L_bottom_right = np.zeros((B, p-k, p-k))
  L_bottom_right[:, np.tri(p-k, dtype=bool, k=-1)] = normals[: , int(k *(k-1)/2) + (p-k)*k :]
  i, j =  np.diag_indices(p-k)
  L_bottom_right[:, i, j] = chi_sqrts[:, :p-k]

  Sigma_hat_chol = np.zeros((B, p, p))
  Sigma_hat_chol[:, :k, :k] = C_chol[np.newaxis, :, :] @ L_top_left
  
  if hasattr(D, "__len__"):
    Sigma_hat_chol[:, k:, :k] = W[np.newaxis, :, :] @ Sigma_hat_chol[:, :k, :k] + D_sqrt[np.newaxis, np.newaxis, :] * L_bottom_left
    Sigma_hat_chol[:, k:, k:] =  D_sqrt[np.newaxis, np.newaxis, :] * L_bottom_right
  else:
    if rho is None:
      Sigma_hat_chol[:, k:, :k] = W[np.newaxis, :, :] @ Sigma_hat_chol[:, :k, :k] + D_sqrt * L_bottom_left
      Sigma_hat_chol[:, k:, k:] =  D_sqrt * L_bottom_right
    else:
      D_chol = get_equicorrelated_chol(p-k, rho, D)
      Sigma_hat_chol[:, k:, :k] = W[np.newaxis, :, :] @ Sigma_hat_chol[:, :k, :k] + D_chol[np.newaxis, :, :] @ L_bottom_left
      Sigma_hat_chol[:, k:, k:] =  D_chol[np.newaxis, :, :] @ L_bottom_right
  Sigma_hat = 1/n * Sigma_hat_chol @ np.transpose(Sigma_hat_chol, (0, 2, 1))
  
  if S is not None:
    S = np.sort(list(S))
    S_comp = complement(p, S)
    perm_3d_inplace(Sigma_hat, np.range(p), np.concatenate([S, S_comp]))
  return np.squeeze(Sigma_hat) if squeeze else Sigma_hat

def get_equicorrelated_chol(k, off_diag, diag=1):
  chol = np.sqrt(diag-off_diag) * np.eye(k)
  cholupdate(chol, np.sqrt(off_diag) * np.ones(k))
  return chol.T

def get_block_W(p, k , num_blocks, block_size, overlap, SNR, seed=None):
  if seed is not None:
    np.random.seed(seed)
  W = np.zeros((p-k, k))
  row_start = 0
  col_start = 0
  row_move = int((p-k)/num_blocks)
  col_move = block_size - overlap
  s = SNR/(SNR + 1)
  
  for i in range(num_blocks):
    if i < num_blocks -1:
      W[row_start: row_start + row_move, col_start : col_start + block_size ] = np.sqrt(s/block_size)
      if seed is not None:
        W[row_start: row_start + row_move, col_start : col_start + block_size] *= np.random.choice(np.array([-1, 1]), (row_move, block_size), replace=True)

    else:
      W[row_start:, col_start:] = np.sqrt(s/(k - col_start))
      if seed is not None:
        W[row_start:, col_start:] *= np.random.choice(np.array([-1, 1]), (p - k - row_start, k - col_start), replace=True)
    
    row_start += row_move
    col_start += col_move
  
  return W

Generate Benchmark Dataset

In [26]:
p = 4000
p_to_block_structure = {p : {'num_blocks' : 7 ,  'block_size': 6, 'overlap': 2}}
k= 30
rho = 0.25
SNR = 2

num_blocks = p_to_block_structure[p]['num_blocks']
block_size = p_to_block_structure[p]['block_size']
overlap= p_to_block_structure[p]['overlap']
C_chol = get_equicorrelated_chol(k, rho)
W = get_block_W(p, k , num_blocks, block_size, overlap, SNR, seed=0)
D = 1/(1 + SNR)

In [28]:
np.random.seed(0)
Sigma_hat = generate_subset_factor_sample_cov(5000, C_chol, W, D, B=None)

In [29]:
CSS = ColumnSubsetSelection()
start = time.time()
CSS.select_subset_from_cov(Sigma_hat, k=30, method='greedy')
end = time.time()
print(end-start)

2.3004393577575684


In [30]:
from pycss import css_reg

start = time.time()
james_out = css_reg(Sigma_hat, 30)
end = time.time()
print(end-start)

0.8628983497619629


In [32]:
np.sort(np.array(list(CSS.S))), np.sort(james_out)

(array([   0,    1,    2,    3,    4,    5,    6,    8,    9,   10,   11,
          12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   23,
          24,   25,   26,   27,   28,   29, 1043, 3055]),
 array([   0,    1,    2,    3,    4,    5,    6,    8,    9,   10,   11,
          12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   23,
          24,   25,   26,   27,   28,   29, 1043, 3055]))